In [ ]:
!pip install konlpy
!pip install gensim
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 34.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=37898fa6c3b59cdda8fdca6dde4a1756f0fa23a444efb112e6af2cb3e3fceaab
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
import os
import pandas as pd
import csv
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from konlpy.tag import Okt
from tqdm import tqdm
from gensim.models import Word2Vec
import time
import random
import numpy as np

# 랜덤 시드 설정
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# 텍스트 전처리 함수 정의
def preprocess_text(text, okt, stop_words):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    tokens = okt.pos(text) # 토큰화 후 형태소 단위로 분석, [(단어, 품사)] 리턴
    tokens = [word for word, pos in tokens if pos in ['Josa', 'Noun', 'Verb']] # 명사, 동사, 조사 이외의 단어들은 제거
    tokens = [word for word in tokens if word not in stop_words] # 불용어 제거
    return tokens

# CSV 파일 읽어서 저장 기사 Content 튜플
def read_csv_file(file_path):
    titles = []
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            titles.append(row['content'])
    return titles

# 전처리된 제목 리스트 생성 함수 정의, tqdm으로 시각화해서 token화된 list 반환
def preprocess_titles(titles, okt, stop_words):
    return [preprocess_text(title, okt, stop_words) for title in tqdm(titles, desc="Processing titles")]

# Word2Vec 모델 학습 및 저장 함수 정의
def train_and_save_word2vec_model(processed_titles, model_path, seed=SEED):
    # 디렉터리가 존재하지 않으면 생성
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    model = Word2Vec(sentences=processed_titles, vector_size=100, window=5, min_count=1, workers=4, sg=1, seed=seed)
    model.save(model_path)
    return model

# 저장된 Word2Vec 모델 불러오기 함수 정의
def load_word2vec_model(model_path):
    return Word2Vec.load(model_path)

from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, HashingTF, IDF

def calculate_tfidf_spark(processed_titles_str):
    # SparkSession 생성
    spark = SparkSession.builder \
        .appName("TF-IDF Calculation") \
        .getOrCreate()

    # DataFrame 생성
    df = spark.createDataFrame([(title,) for title in processed_titles_str], ["titles"])

    # 토큰화
    tokenizer = Tokenizer(inputCol="titles", outputCol="words")
    wordsData = tokenizer.transform(df)

    # TF 계산
    hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=10000)
    featurizedData = hashingTF.transform(wordsData)

    # IDF 계산
    idf = IDF(inputCol="rawFeatures", outputCol="features")
    idfModel = idf.fit(featurizedData)
    rescaledData = idfModel.transform(featurizedData)

    # TF-IDF 결과 추출
    feature_names = hashingTF.getOrDefault('outputCol')
    dtm = rescaledData.select("features").rdd.map(lambda x: x[0])

    # SparkSession 종료
    spark.stop()

    return feature_names, dtm

# TF-IDF 결과 출력 함수 정의 -> 문서에 tf-idf 결과 확인용
def print_tfidf_results(feature_names, dtm, num_docs=10):
    for i in range(num_docs):
        print(f"문서 {i+1}의 TF-IDF 결과:")
        doc = dtm.getrow(i).tocoo()  # 희소행렬의 i번째 행을 COO 포맷으로 변환
        sorted_phrase_scores = sorted(zip(doc.col, doc.data), key=lambda t: t[1], reverse=True)
        for phrase, score in sorted_phrase_scores:
            print(f"  {feature_names[phrase]}: {score}")
        print("\n")

# 입력 단어에 대한 유사 단어 찾기 및 TF-IDF 값 계산 함수 정의
def find_similar_words(word2vec_model, input_word, feature_names, dtm):
    if input_word not in word2vec_model.wv:
        print(f"단어 '{input_word}'가 Word2Vec 모델의 어휘에 없습니다.")
        return {}
    similar_words = word2vec_model.wv.most_similar(input_word, topn=10)
    similar_words_tf_idf = {}
    for word, similarity in similar_words:
        if word in feature_names:
            index = feature_names.tolist().index(word) # 해당 단어의 열 위치
            # 희소행렬에서 해당 단어의 TF-IDF 점수를 가져오기
            tf_idf_scores = dtm[:, index].toarray().flatten() # 해당 단어가 있는 모든 문서에 대해 가져옴
            similar_words_tf_idf[word] = tf_idf_scores
    return similar_words_tf_idf

# 유사 단어 및 TF-IDF 값 출력 함수 정의
def print_similar_words_tf_idf(similar_words_tf_idf):
    for word, scores in similar_words_tf_idf.items():
        print(f"단어: {word}")
        # 0이 아닌 TF-IDF 점수와 해당 문서 인덱스 출력
        non_zero_scores = [(idx, score) for idx, score in enumerate(scores) if score > 0]
        if non_zero_scores:  # 0이 아닌 점수가 있는 경우에만 출력
            print("TF-IDF 점수:")
            for idx, score in non_zero_scores:
                print(f"  문서 {idx + 2}: {score}")
        else:
            print("해당 단어의 TF-IDF 점수가 모든 문서에서 0입니다.")
        print("\n")

In [ ]:
def main(train_model=True):
    # 한국어 불용어 리스트 정의
    stop_words = set([
        '의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를',
        '으로', '자', '에', '와', '한', '하다', '에서', '에게', '이다', '위해'
    ])

    # 한국어 형태소 분석기 Okt 객체 생성
    okt = Okt()

    # 측정 시작
    start_time = time.time()

    # CSV 파일 읽어서 저장 기사 Content
    csv_file_path = '/content/drive/MyDrive/분시컴/result/hani_news.csv'
    titles = read_csv_file(csv_file_path)

    # 전처리된 제목 리스트 생성
    processed_titles = preprocess_titles(titles, okt, stop_words)

    # 모델 경로 설정
    model_path = '/content/drive/MyDrive/분시컴/model/word2vec.model'

    if train_model:
        # 생성된 제목 리스트로 Word2Vec 모델 학습 및 저장
        word2vec_model = train_and_save_word2vec_model(processed_titles, model_path, seed=SEED)
    else:
        # 저장된 Word2Vec 모델 불러오기
        word2vec_model = load_word2vec_model(model_path)

    start_time = time.time()

    # TF-IDF 벡터화 및 밀집 행렬 생성, 가공처리해주어야함
    processed_titles_str = [' '.join(tokens) for tokens in processed_titles]
    feature_names, dtm = calculate_tfidf(processed_titles_str)

    # TF-IDF 결과 출력 (예제: 첫 10개 문서)
    print_tfidf_results(feature_names, dtm, num_docs=10)

    end_time = time.time()
    print(f"총 걸린 시간: {end_time - start_time} 초")

    while True:
      # 사용자로부터 단어 입력 받기
      input_word = input("단어를 입력하세요 (exit을 입력하면 종료됩니다): ")

      # exit을 입력하면 루프 종료
      if input_word == 'exit':
          break

      # 유사한 단어 찾기
      similar_words = find_similar_words(word2vec_model, input_word, feature_names, dtm)

      # 유사 단어 출력
      print_similar_words_tf_idf(similar_words)

    end_time = time.time();
    print(f"총 걸린 시간: {end_time - start_time} 초")

if __name__ == "__main__":
    #main(train_model=True)  # 처음 실행 시 모델 학습 및 저장
    main(train_model=False)  # 이후 실행 시 저장된 모델 불러오기

Processing titles: 100%|██████████| 31362/31362 [49:28<00:00, 10.56it/s]


feature_name: {0} ['가가' '가가린' '가가연' ... '힙합' '힛트' '힝클리']
dtm: {0}   (0, 13447)	0.008969637635659819
  (0, 65054)	0.05017372222417145
  (0, 29438)	0.0350365094491544
  (0, 36386)	0.025411375710202178
  (0, 95880)	0.019851449936597675
  (0, 98230)	0.027756806884545446
  (0, 130057)	0.04915450169404147
  (0, 69285)	0.0705269851262385
  (0, 89149)	0.06275322023452486
  (0, 143429)	0.022552354370228136
  (0, 138541)	0.03506001772905938
  (0, 111030)	0.0317019749608657
  (0, 51036)	0.022923774154105254
  (0, 88261)	0.031051575036730502
  (0, 6627)	0.04059219205228902
  (0, 138721)	0.04761014257252223
  (0, 138023)	0.04478747732103577
  (0, 29778)	0.06166245222179152
  (0, 39194)	0.042075313896270866
  (0, 38483)	0.032016514999068685
  (0, 79434)	0.052797919317344545
  (0, 122133)	0.022358044989989407
  (0, 119777)	0.02660934235580961
  (0, 40933)	0.01663078809616789
  (0, 138862)	0.03367440863642338
  :	:
  (31361, 32236)	0.052511949573963844
  (31361, 141935)	0.020452348534774192
  (31361, 

In [ ]:
# csv 기사내용 확인용
arr = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05557454882925253, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2068688369780905, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07918894015744724, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
print(len(arr))

non_zero_indices = [index for index, value in enumerate(arr) if value != 0]
print(non_zero_indices)


1234
[917, 926, 1154]
